## 3. Simulate 3D topographic residual phase from existing demErr.h5 file ##

In [1]:
%matplotlib inline
import os
import numpy as np
from mintpy.objects import timeseries, sensor
from mintpy.utils import readfile, writefile
from mintpy.simulation import simulation as sim
from mintpy.simulation.plot import *

work_dir = os.path.expanduser('~/development/python/Yunjun_et_al-2019')
os.chdir(work_dir)
print('Go to directory: '+work_dir)

# reference timeseries file for temp/perp baselines
proj_dir = os.path.expanduser('~/insarlab/Galapagos/AlcedoSenDT128/mintpy')
ts_file      = os.path.join(proj_dir, 'timeseries.h5')
geom_file    = os.path.join(proj_dir, 'inputs/geometryRadar.h5')
dem_err_file = os.path.join(proj_dir, 'demErr.h5')
sensorName = sensor.project_name2sensor_name(ts_file)[0]

ts_obj = timeseries(ts_file)
ts_obj.open()
pbase = np.array(ts_obj.pbase)
pbase -= pbase[0]
num_date = ts_obj.numDate
length = ts_obj.length
width = ts_obj.width

out_file = 'sim_topoResid.h5'

Go to directory: /Users/yunjunz/development/python/Yunjun_et_al-2019
open timeseries file: timeseries.h5


In [2]:
## 3D topographic residual
if not os.path.isfile(out_file):
    # read geometry
    inc_angle = readfile.read(geom_file, datasetName='incidenceAngle')[0]
    rg_dist   = readfile.read(geom_file, datasetName='slantRangeDistance')[0]
    pbase *= 10. # amplify the noise
    # calculate topo resid
    dem_err = readfile.read(dem_err_file)[0]
    ts_data = np.zeros((num_date, length, width), np.float32)
    for i in range(num_date):
        ts_data[i,:,:] = dem_err * pbase[i] / (rg_dist * np.sin(inc_angle * np.pi / 180.))
    # write to file
    writefile.write(ts_data, out_file=out_file, ref_file=ts_file)
    print('write simulated topographic residual time-series to file', out_file)
print('Done.')

create timeseries HDF5 file: sim_topoResid.h5 with w mode
create dataset /timeseries of float32    in size of (98, 646, 729) with compression=None
create dataset /dates      of |S8        in size of (98,)
create dataset /bperp      of float32    in size of (98,)
finished writing to sim_topoResid.h5
write simulated topographic residual time-series to file sim_topoResid.h5
Done.


In [4]:
!view.py {out_file} -n 4 --nodisplay

run view.py in MintPy release version v1.1.1-12, release date 2019-05-10
input file is timeseries file: /Users/yunjunz/development/python/Yunjun_et_al-2019/sim_topoResid.h5 in float32 format
file size in y/x: (646, 729)
num of datasets in file sim_topoResid.h5: 98
datasets to exclude (0):
[]
datasets to display (1):
['timeseries-20150331']
data   coverage in y/x: (0, 0, 729, 646)
subset coverage in y/x: (0, 0, 729, 646)
data   coverage in lat/lon: None
subset coverage in lat/lon: None
------------------------------------------------------------------------
colormap: jet
DESCENDING orbit -> flip left-right
figure title: timeseries-20150331
figure size : [8.463622291021672, 6.0]
reading data ...
data    range: [-6.7311964, 8.208577] cm
display range: [-6.7311964, 8.208577] cm
display data in transparency: 1.0
plotting in Y/X coordinate ...
plotting Data ...
plot reference point
flip figure left and right
save figure to timeseries-20150331.png with dpi=300
